In [1]:
import os
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class PrepareTrainingConfig:
  root_dir: Path
  trained_model_path: Path
  updated_base_model_path: Path
  training_data: Path
  params_epochs: int
  params_batch_size: int
  params_learning_rate: int
  params_is_augmentation: bool
  params_image_size: list

In [4]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params =read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_training_config(self) ->PrepareTrainingConfig:
        config =self.config.training
        params=self.params

        create_directories([Path(config.root_dir)])

        prepare_training_config=PrepareTrainingConfig(
            root_dir=Path(config.root_dir),
            trained_model_path=Path(config.trained_model_path),
            updated_base_model_path= Path(self.config.prepare_base_model.updated_base_model_path),
            training_data= Path(os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")),
            params_epochs= params.EPOCHs,
            params_batch_size= params.BATCH_SIZE,
            params_learning_rate= params.LEARNING_RATE,
            params_is_augmentation= params.IS_AUGMENTATION,
            params_image_size= params.IMAGE_SIZE
        )

        return prepare_training_config

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
from numpy.random.mtrand import shuffle
import tensorflow as tf
import time
class PrepareTraining:
    def __init__(self,config:PrepareTrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        return self.model
    
    def train_valid_generator(self):

        datagenerator_kwargs=dict(
            rescale= 1./255,
            validation_split=0.20
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"   
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=40,
            horizontal_flip=True,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [ ]:
try:
    config=ConfigurationManager()
    prepare_Callback_config=config.get_prepare_callback_config()
    prepare_Callback =PrepareCallback(config=prepare_Callback_config)
    callback_list=prepare_Callback.get_tb_ckpt_callbacks()

    prepare_training_config=config.get_prepare_training_config()
    prepare_training =PrepareTraining(config=prepare_training_config)
    prepare_training.get_base_model()
    prepare_training.train_valid_generator()
    prepare_training.train(
        callback_list=callback_list
        )
except Exception as e:
    raise e